In [1]:
import requests
import xmltodict
import json
from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
url = 'https://openapi.gg.go.kr/GgHosptlM'

In [3]:
api_key = '557a4f86f7064f3aad5ef136c0efb6dc'

In [4]:
query = {
    'KEY': api_key, 
    'pSize': 100, 
    'pIndex': 1, 
    'Type': 'xml', 
}

In [5]:
res = requests.get(url = url, params = query)

In [6]:
res.status_code

200

In [7]:
res.headers['content-type']

'application/xml;charset=UTF-8'

In [8]:
res.text
type(res.text)

str

In [9]:
# xml -> dict
dat = xmltodict.parse(res.text)
type(dat)

dict

In [10]:
pprint(dat)

{'GgHosptlM': {'head': {'RESULT': {'CODE': 'INFO-000',
                                   'MESSAGE': '정상 처리되었습니다.'},
                        'api_version': '1.0v',
                        'list_total_count': '1446'},
               'row': [{'APPONT_CANCL_DE': None,
                        'BIZCOND_DIV_NM_INFO': '병원',
                        'BIZPLC_NM': '뉴그린병원',
                        'BSN_STATE_DIV_CD': '99',
                        'BSN_STATE_NM': '삭제',
                        'CLSBIZ_DE': None,
                        'EASING_MEDCARE_APPONT_FORM': None,
                        'EASING_MEDCARE_CHARGE_DEPT_NM': None,
                        'FIRST_APPONT_DE': None,
                        'GENRL_AMBLNC_CNT': '0',
                        'HOSPTLRM_CNT': '28',
                        'LICENSG_CANCL_DE': '2012-09-14',
                        'LICENSG_DE': '2010-12-06',
                        'LOCPLC_AR_INFO': '3536.52',
                        'LOCPLC_FACLT_TELNO': '682-8283',
        

In [12]:
totalCount = int(dat['GgHosptlM']['head']['list_total_count'])
totalCount

1446

In [13]:
rows = 100
pages = int(totalCount / rows) + 1
pages

15

In [15]:
df = pd.DataFrame(data = dat['GgHosptlM']['row'])
df.shape

(100, 39)

In [16]:
def mprcTotalCount(ymonth, rows = 100):
    url = 'https://openapi.gg.go.kr/GgHosptlM'
    query = {
        'KEY': api_key, 
        'pSize': rows, 
        'pIndex': 1, 
        'Type': 'xml', 
    }
    res = requests.get(url = url, params = query)
    dat = xmltodict.parse(res.text)
    totalCount = int(dat['GgHosptlM']['head']['list_total_count'])
    pages = int(totalCount / rows) + 1
    return totalCount, pages

In [18]:
totalCount, pages = mprcTotalCount(ymonth = '202401')
print(totalCount)
print(pages)

1446
15


In [19]:
def mprcDataFrame(ymonth, rows = 100, page = 1):
    url = 'https://openapi.gg.go.kr/GgHosptlM'
    query = {
        'KEY': api_key, 
        'pSize': rows, 
        'pIndex': 1, 
        'Type': 'xml'
    }
    res = requests.get(url = url, params = query)
    dat = xmltodict.parse(res.text)
    df = pd.DataFrame(data = dat['GgHosptlM']['head']['list_total_count'])
    return df

In [21]:
df = mprcDataFrame(ymonth = '202401')
df.shape

ValueError: DataFrame constructor not properly called!

In [ ]:
dfs = pd.DataFrame()
for page in tqdm(range(pages)):
    page += 1
    df = mprcDataFrame(ymonth = '202401', page = page)
    dfs = pd.concat(objs = [dfs, df])

In [ ]:
dfs.shape

In [ ]:
def mprcDataset(ymonth):
    totalCount, pages = mprcTotalCount(ymonth = ymonth)
    dfs = pd.DataFrame()
    for page in tqdm(range(pages)):
        page += 1
        df = mprcDataFrame(ymonth = ymonth, page = page)
        dfs = pd.concat(objs = [dfs, df])
    return dfs

In [ ]:
df = mprcDataset(ymonth = '202401')
df.shape

In [ ]:
ymonths = range(202401, 202413)
dfs = pd.DataFrame()
for ymonth in tqdm(ymonths):
    df = mprcDataset(ymonth = ymonth)
    dfs = pd.concat(objs = [dfs, df])

In [ ]:
dfs.shape

In [ ]:
dfs

In [ ]:
dfs = dfs.reset_index(drop = True)

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir(path = 'C:\\Users\\user\\Documents\\project\\data')
os.getcwd()

In [ ]:
%time dfs.to_excel('MPRC_2024.xlsx', index = False)

In [ ]:
%time dfs.to_csv('MPRC_2024.csv', index = False, encoding = 'CP949')

In [ ]:
os.listdir()

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.getcwd()

In [ ]:
# 상대 경로: '.'(현재 폴더), '..'(부모 폴더)
os.chdir(path = '../data')
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
os.chdir(path = '../code')
os.listdir()

In [ ]:
os.chdir(path = '../data')
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
%time df = pd.read_excel('MPRC_2024.xlsx')

In [ ]:
%time df = pd.read_csv('MPRC_2024.csv', encoding = 'CP949')

In [ ]:
df.head(n = 3)

In [ ]:
df.tail()

In [ ]:
# 재현 가능한 결과를 얻기 위해 초기값을 고정해줍니다.
df.sample(random_state = 1)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.columns = ['기준년월', '국가코드', '수산물코드', '수출입코드', '국가명',
              '경제블럭명', '수산물명', '수출입명', '중량', '달러']

In [ ]:
df.head()

In [ ]:
# df.drop(columns = ['국가코드', '수산물코드', '수출입코드', '경제블럭명'], inplace = True)
df = df.drop(columns = ['국가코드', '수산물코드', '수출입코드', '경제블럭명'])

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['기준년월'] = df['기준년월'].astype(str)

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['수산물명'].unique()

In [ ]:
df['수산물명'].nunique()

In [ ]:
df['수산물명'].value_counts().sort_index().to_excel('MPRC_List.xlsx')

In [ ]:
df['수산물명'].head()

In [ ]:
# str accessor(접근자): 시리즈 원소에 대해 문자열을 처리하는 특별한 객체
# 문자열을 처리하는 다양한 함수를 가지고 있음(모듈)
# raw-string은 문자열 안에 있는 모든 이스케이프의 기능을 해제합니다.
df['수산물명'] = df['수산물명'].str.replace(pat = r'(\(|\[|\{).+', repl = '', regex = True)

In [ ]:
df['수산물명'].value_counts().sort_index().to_excel('MPRC_List_2.xlsx')

In [ ]:
cond = (df['국가명'] == '일본')
cond

In [ ]:
cond.sum()

In [ ]:
# 불리언 인덱싱: 조건에 맞는 행 또는 열만 선택하기
jpn = df.loc[cond, :]

In [ ]:
jpn.shape

In [ ]:
jpn.head()

In [ ]:
jpn['달러'].sum()

In [ ]:
jpn.loc[jpn['수출입명'] == '수출', '달러'].sum()

In [ ]:
jpn.loc[jpn['수출입명'] == '수입', '달러'].sum()

In [ ]:
jpn.groupby(by = '수출입명')['달러'].sum()

In [ ]:
jpn.groupby(by = '수산물명')['달러'].sum().sort_values()

In [ ]:
jpn.loc[jpn['수출입명'] == '수출', :].groupby(by = '수산물명')['달러'].sum().sort_values()

In [ ]:
jpn.loc[jpn['수출입명'] == '수입', :].groupby(by = '수산물명')['달러'].sum().sort_values()

In [ ]:
jpn.groupby(by = ['수출입명', '수산물명'])['달러'].sum().sort_values()

In [ ]:
pvt = pd.pivot_table(data = jpn, index = '수산물명', columns = '수출입명', 
                     values = '달러', aggfunc = 'sum', fill_value = 0)

In [ ]:
pvt.head()

In [ ]:
pvt.columns

In [ ]:
pvt.index

In [ ]:
pvt = pvt.reset_index()

In [ ]:
pvt.columns.name = None

In [ ]:
pvt.head()

In [ ]:
pvt['무역수지'] = pvt['수출'] - pvt['수입']

In [ ]:
pvt.head()

In [ ]:
pvt.sort_values(by = '무역수지', ascending = False)

In [ ]:
def mprcItemIndex(nation):
    cond = (df['국가명'] == nation)
    nat = df.loc[cond, :]
    pvt = pd.pivot_table(data = nat, index = '수산물명', columns = '수출입명', 
                         values = '달러', aggfunc = 'sum', fill_value = 0)
    pvt = pvt.reset_index()
    pvt.columns.name = None
    pvt['무역수지'] = pvt['수출'] - pvt['수입']
    pvt = pvt.sort_values(by = '무역수지', ascending = False)
    return pvt

In [ ]:
mprcItemIndex(nation = '일본')

In [ ]:
# 수출입 상대 국가 개수를 구해보세요.
df['국가명'].nunique()

In [ ]:
# 인덱스에 '국가명', 컬럼스에 '수출입명'으로 설정하고 '달러' 합계를 피벗 테이블로 생성합니다.
# '수출'에서 '수입'을 뺀 '무역수지'를 추가하고 내림차순 정렬하세요.
pvt = pd.pivot_table(data = df, index = '국가명', columns = '수출입명', 
                     values = '달러', aggfunc = 'sum', fill_value = 0)
pvt = pvt.reset_index()
pvt.columns.name = None
pvt['무역수지'] = pvt['수출'] - pvt['수입']
pvt = pvt.sort_values(by = '무역수지', ascending = False)

In [ ]:
pvt

In [ ]:
mprcItemIndex(nation = '미국')

In [ ]:
mprcItemIndex(nation = '중국')

In [ ]:
mprcItemIndex(nation = '러시아')

In [ ]:
mprcItemIndex(nation = '베트남')

In [ ]:
mprcItemIndex(nation = '노르웨이')

In [ ]:
mprcItemIndex(nation = '페루')

In [ ]:
mprcItemIndex(nation = '태국')

In [ ]:
mprcItemIndex(nation = '프랑스')

In [ ]:
mprcItemIndex(nation = '나이지리아')

In [ ]:
pvt = pd.pivot_table(data = df, index = '기준년월', columns = '수출입명', 
                     values = '달러', aggfunc = 'sum', fill_value = 0)
pvt = pvt.reset_index()
pvt.columns.name = None
pvt['무역수지'] = pvt['수출'] - pvt['수입']
pvt = pvt.sort_values(by = '무역수지', ascending = False)

In [ ]:
pvt

In [ ]:
cond = (df['기준년월'] == '202412')
mon = df.loc[cond, :]
pvt = pd.pivot_table(data = mon, index = '수산물명', columns = '수출입명', 
                     values = '달러', aggfunc = 'sum', fill_value = 0)
pvt = pvt.reset_index()
pvt.columns.name = None
pvt['무역수지'] = pvt['수출'] - pvt['수입']
pvt = pvt.sort_values(by = '무역수지', ascending = False)

In [ ]:
pvt

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.getcwd()

In [ ]:
os.chdir(path = r'C:\Users\user\Documents\project\data')
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
df = pd.read_csv('KBO_Hitter_Stats_2024.csv', encoding = 'CP949')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.tail(n = 10)

In [ ]:
df = df.fillna(value = 0)

In [ ]:
df.info()

In [ ]:
df.groupby(by = '팀명')['선수명'].count().sort_values(ascending = False)

In [ ]:
df.groupby(by = '팀명')['타수'].sum()

In [ ]:
df.groupby(by = '팀명')['안타'].sum()

In [ ]:
pvt = pd.pivot_table(data = df, index = '팀명', values = ['타수', '안타'], 
                     aggfunc = 'sum', sort = False)
pvt = pvt.reset_index()
pvt['타율'] = pvt['안타'] / pvt['타수']
pvt['타율'] = pvt['타율'].round(3)
pvt

In [ ]:
df.groupby(by = '팀명')['홈런'].sum().sort_values(ascending = False)

In [ ]:
df.groupby(by = '팀명')['홈런'].mean().sort_values(ascending = False)

In [ ]:
# 규정타석: 팀 경기수(144) * 3.1
cond = (df['타석'] >= 144*3.1)
df.loc[cond, :].groupby(by = '팀명')['홈런'].sum().sort_values(ascending = False)

In [ ]:
df.loc[cond, :].groupby(by = '팀명')['홈런'].mean().sort_values(ascending = False)

In [ ]:
df.loc[cond, :].groupby(by = '팀명')['홈런'].count().sort_values(ascending = False)

In [ ]:
cond = (df['홈런'] >= 10)
df.loc[cond, '타자유형'] = '슬러거'
df.loc[~cond, '타자유형'] = '똑딱이'
df.sample(n = 5, random_state = 1234)

In [ ]:
df['타자유형'] = np.where(cond, '슬러거', '똑딱이')

In [ ]:
pd.crosstab(index = df['팀명'], columns = df['타자유형'], 
            margins = True, margins_name = '합계', 
            normalize = 'index') * 100

In [ ]:
df['팀명'] == 'NC'

In [ ]:
cond = (df['팀명'] == 'NC') & (df['타자유형'] == '슬러거')
df.loc[cond, '선수명']

In [ ]:
# 선수별 볼삼비(= 볼넷 / 삼진)를 추가하고 볼삼비가 높은 순으로 내림차순 정렬해서
# 상위 10명의 선수를 확인해보세요. 단, 타수가 100 이상인 선수만으로 실행합니다.
df['볼삼비'] = df['볼넷'] / df['삼진']
cond = df['타수'] >= 100
cols = ['팀명', '선수명', '볼넷', '삼진', '볼삼비']
df.loc[cond, cols].sort_values(by = '볼삼비', ascending = False).head(n = 10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rc(group = 'font', family = 'Malgun Gothic', size = 10)

In [ ]:
pvt = pvt.sort_values(by = '타율', ascending = False)
pvt

In [ ]:
for i in enumerate(pvt['타율']):
    print(i)

In [ ]:
sns.barplot(data = pvt, x = '팀명', y = '타율', facecolor = '0.8', edgecolor = '0')
plt.xlim(-1, 10)
plt.ylim(0, 0.35)
for i, v in enumerate(pvt['타율']):
    plt.text(x = i, y = v - 0.006, s = f'{v:.3f}', ha = 'center', va = 'top', 
             fontsize = 8, fontweight = 'bold', color = 'red')
plt.title(label = '2024 KBO 팀타율')
plt.savefig(fname = 'test.png')
plt.show()

In [ ]:
df['타율'].min()

In [ ]:
df['타율'].max()

In [ ]:
df['타율'].agg(func = ['min', 'max'])

In [ ]:
# sns.histplot(data = df, x = '타율', binrange = (0, 0.7), bins = 14)
sns.histplot(data = df, x = '타율', binrange = (0, 0.7), binwidth = 0.05)
plt.show()

In [ ]:
df['타수'].agg(func = ['min', 'max'])

In [ ]:
cond = (df['팀명'] == 'KIA') | (df['팀명'] == '삼성')

In [ ]:
sns.histplot(data = df.loc[cond, :], x = '타수', binrange = (0, 600), binwidth = 50, hue = '팀명')
plt.show()

In [ ]:
plt.rc(group = 'axes', unicode_minus = False)

In [ ]:
cond = df['팀명'].isin(values = ['KIA', '한화'])

In [ ]:
sns.kdeplot(data = df.loc[cond, :], x = '타수', hue = '팀명', fill = True)
plt.show()

In [ ]:
cond = df['타석'] >= 144*3.1
sns.scatterplot(data = df.loc[cond, :], x = '출루율', y = '장타율')
plt.axvline(x = df.loc[cond, '출루율'].mean(), color = 'red', linewidth = 0.5, linestyle = '--')
plt.axhline(y = df.loc[cond, '장타율'].mean(), color = 'red', linewidth = 0.5, linestyle = '--')
plt.show()